# NLP分类任务评价指标
## 1.宏平均（Macro）与微平均（Micro）以及加权平均
>在多类别（multi-class）或多标签（multi-label）分类任务中，我们需要将各类别的指标汇总得到整体性能。宏平均 (Macro-average) 和 微平均 (Micro-average) 是两种常用的汇总方法。
### (0)
对一个有K个类别的分类任务。把每个类别c都当成“一对多(one-vs-rest)”的二分类来数：
- TPc:真实是c，预测也是c
- NPc:真实不是c，但预测是c
- FNc:真实是c，预测不是c

于是对于每个类的指标：
- $P_{c}=\frac{T P_{c}}{T P_{c}+F P_{c}}$
- $R_{c}=\frac{T P_{c}}{T P_{c}+F N_{c}}$
- $F 1_{c}=\frac{2 P_{c} R_{c}}{P_{c}+R_{c}}=\frac{2 T P_{c}}{2 T P_{c}+F P_{c}+F N_{c}}$

定义每个类的真实样本数（support）：$s_{c}=T P_{c}+F N_{c}$
### (1)宏平均（Macro）
>对每个类别分别计算Precision、Recall、F1等指标，然后对各类别的指标直接取算术平均。

公式：
- $P_{\text {macro }}=\frac{1}{K} \sum_{c=1}^{K} P_{c}$
- $R_{\text {macro }}=\frac{1}{K} \sum_{c=1}^{K} R_{c}$
- $F 1_{\text {macro }}=\frac{1}{K} \sum_{c=1}^{K} F 1_{c}$

宏平均赋予每个类别相同的权重，不考虑类别的不均衡。因此，宏平均能够平等地看待每个类别，但稀有类别的表现会对宏平均值产生显著影响。宏平均适用于关注每个类别表现、希望评价模型对小类别是否也有良好效果的场景。
### (2)微平均（Micro）
>先汇总所有类别的TP、FP、FN，再计算Precision、Recall等。

先汇总：

$T P_{\text {micro }}=\sum_{c} T P_{c}, F P_{\text {micro }}=\sum_{c} F P_{c}, F N_{\text {micro }}=\sum_{c} F N_{c}$

再计算：
- $P_{\mathrm{micro}}=\frac{T P_{\mathrm{micro}}}{T P_{\mathrm{micro}}+F P_{\mathrm{micro}}}$
- $R_{\mathrm{micro}}=\frac{T P_{\mathrm{micro}}}{T P_{\mathrm{micro}}+F N_{\mathrm{micro}}}$
- $F 1_{\text {micro }}=\frac{2 T P_{\text {micro }}}{2 T P_{\text {micro }}+F P_{\text {micro }}+F N_{\text {micro }}}$

在单标签多分类（每个样本只有一个真类，预测也只给一个类）里：
- $\begin{array}{l}
\sum_{c} F P_{c}=\sum_{c} F N_{c} \\
P_{\text {micro }}=R_{\text {micro }}=F 1_{\text {micro }}=\text { Accuracy }
\end{array}$

这相当于每个样本同等权重，不论其属于哪个类别。对于多类别问题，如果计算所有类别整体的Micro Precision/Recall，会等同于计算总体准确率 (Accuracy)。Micro的优点是受到大类别的主导，更能反映模型对整个数据集的性能。当评价模型整体效果或在类别分布较为平衡时，Micro平均比较有意义。但在类别极不平衡时，Micro由于被大量样本的主流类别主导，可能掩盖少数类的表现。
### (3)加权平均（Weighted）
>加权平均可以看作是对宏平均的扩展，它对每个类别的指标按该类别的支持度（support，即真实样本数）加权后再求平均。

公式：
- $P_{\text {weighted }}=\frac{\sum_{c} s_{c} P_{c}}{\sum_{c} s_{c}}, \quad R_{\text {weighted }}=\frac{\sum_{c} s_{c} R_{c}}{\sum_{c} s_{c}}, \quad F_{1 \text { weighted }}=\frac{\sum_{c} s_{c} F 1_{c}}{\sum_{c} s_{c}}$

这样大的类别对平均值影响更大，小的类别影响更小。Weighted average在类别不平衡时提供了一种折中：既考虑各类别表现，又根据样本数调整权重。Scikit-learn的classification_report通常同时报告宏平均和加权平均，方便比较。

### 直觉
Macro：每个类一样重要
>小类表现差 → macro 掉得很厉害（“公平但严格”）

Micro：每个样本一样重要
>大类样本多 → micro 更像大类指标（“总体但容易被大类支配”）

Weighted：按类样本数给权重
>大类影响大，但小类仍会贡献一些（“折中”）

## 2.代码实践：评价指标计算与不平衡场景分析
下面我们通过一个文本分类的示例来实践上述概念。我们将使用Python和scikit-learn构建一个简单的文本分类器，对模拟的数据进行训练和评估。此示例涵盖以下内容：
- 数据集与分类器：构造一个简单的三分类数据集（模拟文本），训练一个Logistic Regression分类器。
- 混淆矩阵：输出混淆矩阵的数值和热力图。
- 指标计算：计算每个类别的Precision、Recall、F1，以及macro平均和weighted平均的值。
- 不平衡分析：模拟类别不平衡，对比不同指标在不平衡场景下的表现。

### (1)数据集准备与模型训练
首先，我们准备一个模拟的文本数据集。假设有3个类别：Class0, Class1, Class2。我们人为生成一些单词来代表不同类别的文本特征，例如水果相关词对应Class0，动物相关词对应Class1，科技相关词对应Class2。为了模拟混淆和不平衡，我们让各类别的文本中混入一些其他类别的词作为噪音，并让Class2的样本数量远少于另外两个类别。下面是数据生成和模型训练的代码：

In [1]:
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# 定义三类词汇列表
fruit_words = ["apple", "orange", "banana", "grape", "berry"]
animal_words = ["dog", "cat", "fish", "bird", "lion"]
tech_words  = ["computer", "software", "hardware", "internet", "AI"]
common_words = ["the", "and", "is", "in", "on"]

# 函数：生成一条随机文本
def generate_sentence(main_words, other_words1, other_words2):
    words = []
    # 每条文本包含main类词汇和其他类词汇的混合
    words += random.choices(main_words, k=random.randint(1,3))
    words += random.choices(other_words1, k=random.randint(1,3))
    words += random.choices(other_words2, k=random.randint(1,3))
    words += random.choices(common_words, k=random.randint(0,2))
    random.shuffle(words)
    return " ".join(words)

# 准备训练集（不平衡：Class0=80, Class1=80, Class2=20）
X_train, y_train = [], []
for _ in range(80):
    X_train.append(generate_sentence(fruit_words, animal_words, tech_words))
    y_train.append(0)   # Class0
for _ in range(80):
    X_train.append(generate_sentence(animal_words, fruit_words, tech_words))
    y_train.append(1)   # Class1
for _ in range(20):
    X_train.append(generate_sentence(tech_words, fruit_words, animal_words))
    y_train.append(2)   # Class2

# 准备测试集（Class0=40, Class1=40, Class2=10）
X_test, y_test = [], []
for _ in range(40):
    X_test.append(generate_sentence(fruit_words, animal_words, tech_words))
    y_test.append(0)
for _ in range(40):
    X_test.append(generate_sentence(animal_words, fruit_words, tech_words))
    y_test.append(1)
for _ in range(10):
    X_test.append(generate_sentence(tech_words, fruit_words, animal_words))
    y_test.append(2)

# 构建 TF-IDF + 逻辑回归模型的Pipeline并训练
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000))
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LogisticRegression(max_iter=1000))])

### (2)模型性能评估：混淆矩阵和分类报告
我们使用混淆矩阵和classification report来评估模型：

In [2]:
from sklearn.metrics import confusion_matrix, classification_report

# 模型预测
y_pred = pipeline.predict(X_test)

# 打印分类报告（Precision/Recall/F1）和混淆矩阵
print(classification_report(y_test, y_pred, digits=3, zero_division=0))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0      0.419     0.450     0.434        40
           1      0.435     0.500     0.465        40
           2      0.000     0.000     0.000        10

    accuracy                          0.422        90
   macro avg      0.284     0.317     0.300        90
weighted avg      0.379     0.422     0.399        90

Confusion Matrix:
[[18 22  0]
 [19 20  1]
 [ 6  4  0]]


从上面的分类报告可以解读出以下关键信息：

- 类别0（Class0）的精确率为41.9%，召回率45.0%，F1约43.4%。在40个实际属于Class0的样本中模型识别出45%，而预测为Class0的样本中只有41.9%是正确的。

- 类别1的精确率43.5%，召回率50.0%，F1约46.5%，性能略好于类别0。

- 类别2的精确率和召回率均为0%，F1也为0，意味着模型完全未能识别出任何Class2的样本（在预测中从未出现类别2）。这是由于类别2样本非常少，且可能特征不明显，模型倾向于将它们错分为其他类别。

- 准确率 (accuracy) 为42.2%，即90个测试样本中有约42%被正确分类。

- 宏平均 (macro avg) 的Precision约28.4%，Recall约31.7%，F1约30.0%，明显低于准确率，这反映了模型在少数类（Class2）上的糟糕表现拉低了总体表现。

- 加权平均 (weighted avg) 的F1约39.9%，介于accuracy和macro F1之间，表明加权平均受类别0和1（大类）的影响更大，但少数类的差表现仍有所反映。

从混淆矩阵来看（行表示实际类别，列表示预测类别）：

模型将实际属于Class2的10个样本错误地预测为Class0或Class1（6个被当作0类，4个被当作1类），没有一个被预测为Class2。这与分类报告中Class2的召回率0相符。对于Class0和Class1，模型也存在不少混淆，例如实际40个Class0中有22个被错分为Class1，而实际40个Class1中有19个被错分为Class0。